In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
     def __init__(self):
         super().__init__()
         self.conv1 = nn.Conv2d(3, 6, 5)
         self.conv1_bn = nn.BatchNorm2d(6)
         self.pool = nn.MaxPool2d(2, 2)
         self.conv2 = nn.Conv2d(6, 8, 9)
         self.conv2_bn = nn.BatchNorm2d(8)
         self.fc1 = nn.Linear(8 * 3 * 3, 30)
         self.fc1_bn = nn.BatchNorm1d(30)
         self.fc2 = nn.Linear(30, 20)
         self.fc2_bn = nn.BatchNorm1d(20)
         self.fc3 = nn.Linear(20, 10)

     def forward(self, x):
         x = self.pool(F.elu( self.conv1_bn(self.conv1(x))  ))
         x = self.pool(F.elu( self.conv2_bn(self.conv2(x))  ))
         x = torch.flatten(x, 1) # flatten all dimensions except batch
         x = F.elu( self.fc1_bn(self.fc1(x)) )
         x = F.elu( self.fc2_bn(self.fc2(x)) )
         x = self.fc3(x)
         return x

# fix model parameters
torch.manual_seed(0)


net = Net()

In [3]:
transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 1000
trainset = torchvision.datasets.CIFAR10(root='/home/buyun/Documents/GitHub/PyGRANSO/examples', train=True, download=True, transform=transform)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=False, num_workers=2)
# data_in
for i, data in enumerate(trainloader, 0):        
    if i >= 1:
         break   
    # get the inputs; data is a list of [inputs, labels]
    inputs, labels = data



Files already downloaded and verified


In [4]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
# optimizer = torch.optim.LBFGS(net.parameters(), lr=1, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn=None)
# optimizer = torch.optim.LBFGS(net.parameters(), lr=0.01, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn="strong_wolfe")
optimizer = torch.optim.LBFGS(net.parameters(), lr=1, max_iter=20, max_eval=None, tolerance_grad=1e-07, tolerance_change=1e-09, history_size=100, line_search_fn="strong_wolfe")



for i in range(20):
#     # zero the parameter gradients
#     optimizer.zero_grad()

#     # forward + backward + optimize
#     outputs = net(inputs)
#     loss = criterion(outputs, labels)

#     loss.backward()
#     optimizer.step(lambda : loss)

    def closure():
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        return loss
    
    loss = optimizer.step(closure)

    if i % 1 == 0:
        print(loss.item())

/home/buyun/anaconda3/envs/cuosqp_pygranso/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /opt/conda/conda-bld/pytorch_1623448255797/work/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


2.3954172134399414
1.352547526359558
0.6083256006240845
0.0171643178910017
1.9431095665822795e-07
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08
1.2159345530449173e-08


In [5]:
outputs = net(inputs)
acc = (outputs.max(1)[1] == labels).sum().item()/labels.size(0)
print("Train acc = {}".format(acc))

Train acc = 1.0
